In [1]:
using Statistics, CSV, DataFrames, Shuffle

In [123]:
# using Pkg
# Pkg.add("Shuffle")

In [2]:
include("utils.jl")
include("synthdid.jl")
include("solver.jl")
include("data.jl")
include("vcov.jl")

vcov_synthdid_estimate (generic function with 1 method)

In [3]:
# include("synthdid.jl")

In [4]:
# function sum_normalize(x)
#     if sum(x) != 0
#         x/sum(x)
#     else
#         repeat([1/length(x)], length(x))
#     end
# end

In [3]:
setup = panel_matrices(data("california_prop99"));

algo = synthdid_estimate(setup.Y, setup.N0, setup.T0, omega_intercept=true);
estimate = algo;
replications = 5;

In [5]:
function placebo_se(estimate, replications)
    setup = estimate.setup
    opts = estimate.opts
    weights = estimate.weight
    N1 = size(setup["Y"])[1] - setup["N0"]
    if setup["N0"] <= N1
        error("must have more controls than treated units to use the placebo se")
    end
    function theta(ind)
        N0 = length(ind)-N1
        weights1 = copy(weights)
        weights_boot = weights1
        weights_boot["omega"] = sum_normalize(weights["omega"][ind[1:N0]]);
        synthdid_estimate(setup["Y"][ind,:], N0, setup["T0"],  X=setup["X"][ind, :,:], weights=weights_boot,
                            zeta_omega = opts["zeta_omega"], zeta_lambda = opts["zeta_lambda"], omega_intercept = opts["omega_intercept"],
                            lambda_intercept = opts["lambda_intercept"], update_omega = opts["update_omega"], update_lambda = opts["update_lambda"],
                            min_decrease = opts["min_decrease"], max_iter = opts["max_iter"])
    end
    
    a = 0
    for i in 1:replications
        if i == 1
            a = theta(shuffle(1:setup["N0"])).estimate
        else
            a = vcat(a,theta(shuffle(1:setup["N0"])).estimate)
        end
    end
    sqrt((replications-1)/replications) * std(a)
end

In [17]:
placebo_se(algo, 5)

5.167860360934321

In [24]:
function vcov_synthdid_estimate(object; method = ["bootstrap", "jackknife", "placebo"], replications = 200)
    if method == "placebo"
        se = placebo_se(object, replications)
    end
    se^2
end

vcov_synthdid_estimate (generic function with 1 method)

In [4]:
vcov_synthdid_estimate(algo, method = "placebo")

85.79277404694083